<a href="https://colab.research.google.com/github/slayerzeroa/Railway_Accident_BERT/blob/main/Tagtog_Preprocess_Entity_Recognitioin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code

In [ ]:
!pip install gspread
!pip install oauth2client
!pip install transformers
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 라이브러리
import torch
from torch import nn, optim, tensor
from torch.utils.data import DataLoader, Dataset,TensorDataset, random_split
import torch.nn.functional as F

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, mean_squared_error

# from transformers import BertTokenizer

import json
import glob
import re
import os
import ast
import html
import requests

import zipfile
import shutil

import nltk
from nltk.tokenize import word_tokenize

import pandas as pd

from google.colab import files

In [ ]:
#punkt 다운로드
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#로그인 위치
url = "https://tagtog.com/-login"

#다운로드 위치
file_url = 'https://www.tagtog.com/slayerzeroa/Railway_BERT/-downloads/dataset-as-anndoc' 
o_file = 'Railway_Annotation.zip'  
if os.path.exists(o_file):
    os.remove(o_file)

#로그인 정보
login_info = {
    'loginid' : 'slayerzeroa@gmail.com',
    'password' : 'gyysxw5rU3NzrYX',
}

#로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)

    with open(o_file,"wb") as output:
        output.write(r.content)

In [ ]:
#압축파일 풀기
folder_path = "./tagtog_relation_extraction/"

zip_ = zipfile.ZipFile("Railway_Annotation.zip")
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
zip_.extractall(folder_path)

In [ ]:
#폴더 경로
folder_name = "./tagtog_relation_extraction/Railway_BERT/"

#name list
context_name_list = os.listdir(folder_name + "plain.html/pool")
relation_name_list = os.listdir(folder_name + "ann.json/master/pool")

context_name_list.sort()
relation_name_list.sort()

#폴더 경로
contexts_folders_paths = [folder_name + "plain.html/pool/" + c for c in context_name_list]
relation_folders_paths = [folder_name + "ann.json/master/pool/" + c for c in relation_name_list]
print(contexts_folders_paths)
print(relation_folders_paths)

#anntation_lenged 정보
annotations_legend = folder_name + "annotations-legend.json"
with open(annotations_legend,"r") as f:
    annotations_legend = json.load(f)


['./tagtog_relation_extraction/Railway_BERT/plain.html/pool/a2nq4ZkAtmxrdM.yECpQx1Ae4rYu-text_4.txt.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/a7njLl5A3qbkk787l1KnKTod9AHy-text_2.txt.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/aFBtrQNRHHFnpD56uimygcqw5tAW-text_1.txt.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/aT0v7lsjlValvCzGFIBlxAutdrrG-text_7.txt.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/aVLI6NO6mNSFompKzlsJc0skVUo8-text_5.txt.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/aXdTlZzcuUIsWDg1KxZzhYiaHUWC-text_6.txt.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt.plain.html', './tagtog_relation_extraction/Railway_BERT/plain.html/pool/agCjSEwp8H0mHv_BeVgYapyRM80m-text_3.txt.plain.html']
['./tagtog_relation_extraction/Railway_BERT/ann.json/master/pool/a2nq4ZkAtmxrdM.yECpQx1Ae4rYu-text_4.txt.

In [ ]:
def remove_tag(html_text):
  check = html_text
  check = re.sub('<title>.*</title>', '', check)
  check = re.sub('(<([^>]+)>)', '', check).strip()
  check = re.sub('\n', '', check)
  check = re.sub(' +', ' ', check)
  check = re.sub('·','', check)
  check = re.sub('○', '', check)
  return check

In [ ]:
text_list = []
label_list = []

for context_path, relation_path in zip(contexts_folders_paths, relation_folders_paths):
  # entity json 파일 열기
  with open(relation_path, "r", encoding="UTF-8") as f:
    content = json.load(f)

  # 텍스트 토큰 대치
  check_entity = dict()
  for i in content['entities']:
    check_entity[i['offsets'][0]['text']] = i['classId']

  # plain text 열고 태그 제거
  with open(context_path, "r", encoding="UTF-8") as g:
    context = g.read()
  removing = remove_tag(context)

  # 워드 토크나이즈, entity 체크
  check = word_tokenize(removing)

  checking_list = []

  for i in check:
    if i in check_entity:
      checking_list.append(annotations_legend[check_entity[i]])
    if i not in check_entity:
      checking_list.append('O')
  text_list.append(' '.join(check))
  label_list.append(' '.join(checking_list))

In [ ]:
# google api key
key = files.upload()

Saving railway-tagtog-d24e6901ddb0.json to railway-tagtog-d24e6901ddb0.json


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = './railway-tagtog-d24e6901ddb0.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo/edit#gid=0"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
## gc.create(spreadsheet_name) # 스프레드시트 생성


#시트 선택하기
sheet_name = "context"
worksheet = doc.worksheet(sheet_name) #해당 시트가 있는 경우 불러오기
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함

In [ ]:
# 시트의 모든 데이터 가져오기
values = worksheet.get_all_values()
header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)
column_list = ["text","label"]
data = data[column_list]
data.head()

worksheet.resize(len(text_list)+1,10)
list_range = f"a2:b{len(text_list)+1}"
cell_list = worksheet.range(list_range)

for i in range(len(cell_list)//len(column_list)):
    cell_list[(2*i)].value = text_list[i]
    cell_list[(2*i)+1].value = label_list[i]
worksheet.update_cells(cell_list)

{'spreadsheetId': '1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo',
 'updatedRange': 'context!A2:B9',
 'updatedRows': 8,
 'updatedColumns': 2,
 'updatedCells': 16}

# Test


In [ ]:
with open('./tagtog_relation_extraction/Railway_BERT/ann.json/master/pool/aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt.ann.json', "r", encoding="UTF-8") as f:
  test = json.load(f)

print(test)
print(test['entities'])

{'annotatable': {'parts': ['s1v1']}, 'anncomplete': False, 'sources': [], 'metas': {}, 'entities': [{'classId': 'e_28', 'part': 's1v1', 'offsets': [{'start': 4, 'text': '차량'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_6', 'part': 's1v1', 'offsets': [{'start': 19, 'text': '대차가'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_23', 'part': 's1v1', 'offsets': [{'start': 23, 'text': '선로'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_22', 'part': 's1v1', 'offsets': [{'start': 31, 'text': '탈선하였다'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}, 'fields': {}, 'normalizations': {}}], 'relations': [{'classId': 'r_25', 'type': 'linked

In [ ]:
check_entity = dict()

for i in test['entities']:
  check_entity[i['offsets'][0]['text']] = i['classId']

print(check_entity)

{'차량': 'e_28', '대차가': 'e_6', '선로': 'e_23', '탈선하였다': 'e_22'}


In [ ]:
with open('/content/tagtog_relation_extraction/Railway_BERT/plain.html/pool/aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt.plain.html', "r") as f:
  sample = f.read()

print(sample)

<!DOCTYPE html >
<html id="aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt" data-origid="text.txt" class="anndoc" data-anndoc-version="3.6" lang="" xml:lang="" xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <meta charset="UTF-8"/>
    <meta name="generator" content="net.tagtog.anndoc.v3.parsers.general.VerbatimParser_v1_0_1"/>
    <title>aYqDitzamGH6gW4K6Tv_k3cHoVgW-text.txt</title>
  </head>
  <body>
    <article>
      <section data-type="">
        <div class="content">
          <pre id="s1v1">5번째 차량 ( 3513호 ) 앞 대차가 선로 우측으로 탈선하였다.</pre>
        </div>
      </section>
    </article>
  </body>
</html>



In [ ]:
removing = remove_tag(sample)
print(removing)

5번째 차량 ( 3513호 ) 앞 대차가 선로 우측으로 탈선하였다.


In [ ]:
check = word_tokenize(removing)

In [ ]:
print(check)

['5번째', '차량', '(', '3513호', ')', '앞', '대차가', '선로', '우측으로', '탈선하였다', '.']


In [ ]:
checking_list = []

for i in check:
  if i in check_entity:
    checking_list.append(annotations_legend[check_entity[i]])
  if i not in check_entity:
    checking_list.append('O')

In [ ]:
print(checking_list)

['O', 'OBJECT', 'O', 'O', 'O', 'O', 'IMPLIED', 'SOLUTION', 'O', 'ACCIDENT', 'O']


In [ ]:
print(' '.join(checking_list))

O OBJECT O O O O IMPLIED SOLUTION O ACCIDENT O
